In [1]:
import math
import cv2
import sys
import os
import numpy as np

In [ ]:
path_to_video = './Monkey.avi'

cap = cv2.VideoCapture(path_to_video)

if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)

time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

while(1):
    return_flag, frame = cap.read()  
    if not return_flag:
       
        print('Video Reach End')
        break
        
  
    cv2.imshow('VideoWindowTitle-Monkey', frame)
  
    

    if cv2.waitKey(30) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
frame_save_path = './frames/'
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [122]:
cap = cv2.VideoCapture(path_to_video)
create_dir_if_not_exists(frame_save_path)
if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)
time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_counter = 0 

while frame_counter <= 150:
    return_flag, frame = cap.read()  
    if not return_flag:
        print('Video Reach End')
        break

    cv2.imshow('VideoWindowTitle-Monkey', frame)
    cv2.imwrite(frame_save_path + 'frame%d.tif' % frame_counter, frame)
    frame_counter += 1
    
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [3]:
def arrowdraw(img, x1, y1, x2, y2):
    radians = math.atan2(x1-x2, y2-y1)
    x11 = 0
    y11 = 0
    x12 = -10
    y12 = -10

    u11 = 0
    v11 = 0
    u12 = 10
    v12 = -10
    
    x11_ = x11*math.cos(radians) - y11*math.sin(radians) + x2
    y11_ = x11*math.sin(radians) + y11*math.cos(radians) + y2

    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2
    
    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2

    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2
    img = cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 2)
    img = cv2.line(img, (int(x11_), int(y11_)), (int(x12_), int(y12_)), 
    (255, 255, 255), 2)
    img = cv2.line(img, (int(u11_), int(v11_)), (int(u12_), int(v12_)), 
    (255, 255, 255), 2)
    
    return img

In [4]:
def getSSD(source_grid, target_grid):

    return np.sqrt(np.sum((source_grid-target_grid)**2))   

In [124]:
count = 0
process_frame = 0
k = 3
grid_size = 2*k+1

t_min = 112
t_max = 150

cap = cv2.VideoCapture('./monkey.avi')
if not cap.isOpened():
    print('monkey.avi not opened')
    sys.exit(1)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

while count <= 0:
    
    oframe = cv2.imread(frame_save_path + 'frame%d.tif' %count)

    
    tframe = cv2.imread(frame_save_path + 'frame%d.tif' %(count+1))

    

    
    for i in range(grid_size*14,int(frame_height-3*grid_size),grid_size):
        for j in range(grid_size*14,int(frame_width-3*grid_size),grid_size):

            source_grid = oframe[i:i+grid_size, j:j+grid_size,:]
#             print("Source grid: ",source_grid[0][0])
            
            
            
            
            

            region = []   
            
            for x in range(i-k*grid_size,i+k*grid_size,grid_size):
                if x > frame_height-grid_size or x <= 0:
                    continue
                
                for y in range(j-k*grid_size,j+k*grid_size,grid_size):
                    if y > frame_width-grid_size or y <= 0:
                        continue
                    
                    target_grid = tframe[x:x+grid_size, y:y+grid_size,:]
                       
                    
#                     print("target_grid: ",target_grid[0][0])
#                     print("x:",x)
#                     print("y:",y)
#                     print("i//grid_size:",i//grid_size)
#                     print("j//grid_size:",j//grid_size)
                    SSD = getSSD(source_grid, target_grid)
                     
                    region.append([SSD,y,x])
                    
                    
                    #print("SSD: \n" ,SSD)
                    
#                     print("Source grid: ",source_grid[0][0])
#                     print("Target grid: \n" ,target_grid[0][0])
                    
            if len(region) == 0:
                continue
            min_SSD = min(region)
#             if min_SSD[0] > t_min:
#                 print("min SSD: " ,min_SSD[0])
#                 print("i//grid_size:",i//grid_size)
#                 print("j//grid_size:",j//grid_size)
#                 print("x//grid_size:",min_SSD[2]//grid_size)
#                 print("y//grid_size:",min_SSD[1]//grid_size)
                
#                 print("x:",x)
#                 print("y:",y)
#                 print("Source grid: ",source_grid[0][0])
#                 print("target_grid: ",target_grid[0][0])
                


            if min_SSD[0] > t_min and min_SSD[0] < t_max:   
                
                arrowdraw(oframe,j,i, min_SSD[1], min_SSD[2])
                

                    
            create_dir_if_not_exists('./output/')  
            cv2.imwrite('output/output%d.tif' % count, oframe)
            

           

    
    #cv2.imshow('Monkey video with arrows', monkey_copy)

    count += 1
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
    process_frame += 1
cap.release()
cv2.destroyAllWindows()


In [125]:
frame_load_path = './output/'
create_dir_if_not_exists(frame_load_path)
path_to_output_video = './Monkey video with vectors.mov'
out = cv2.VideoWriter(path_to_output_video, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (int(frame_width), int(frame_height)))
frame_counter = 0
while(frame_counter <= 149):
    img = cv2.imread(frame_load_path + 'output%d.tif' % frame_counter)
    if img is None:
        print('No more frames to be loaded')
        break;
    out.write(img)
    frame_counter += 1
out.release()
cv2.destroyAllWindows()